In [1]:
import pandas as pd
import numpy as np

In [2]:
TRAIN_DATA = "data/preprocessed/train_data.csv"

In [3]:
train_df = pd.read_csv(TRAIN_DATA)

In [4]:
train_df.shape

(20974, 41)

In [5]:
train_df.isnull().sum()

Price                  0
city                   0
Area                   0
Location               0
No. of Bedrooms        0
Resale                 0
MaintenanceStaff       0
Gymnasium              0
SwimmingPool           0
LandscapedGardens      0
JoggingTrack           0
RainWaterHarvesting    0
IndoorGames            0
ShoppingMall           0
Intercom               0
SportsFacility         0
ATM                    0
ClubHouse              0
School                 0
24X7Security           0
PowerBackup            0
CarParking             0
StaffQuarter           0
Cafeteria              0
MultipurposeRoom       0
Hospital               0
WashingMachine         0
Gasconnection          0
AC                     0
Wifi                   0
Children'splayarea     0
LiftAvailable          0
BED                    0
VaastuCompliant        0
Microwave              0
GolfCourse             0
TV                     0
DiningTable            0
Sofa                   0
Wardrobe               0


In [6]:
train_df.head()

,Price,city,Area,Location,No. of Bedrooms,Resale,MaintenanceStaff,Gymnasium,SwimmingPool,LandscapedGardens,...,LiftAvailable,BED,VaastuCompliant,Microwave,GolfCourse,TV,DiningTable,Sofa,Wardrobe,Stadium
0,11000000,Delhi,1400,Sector 11 Dwarka,2,1,0,0,0,0,...,1,1,0,1,0,1,1,1,0,1
1,4883000,Bangalore,1565,Varthur,3,0,9,9,9,9,...,9,9,9,9,9,9,9,9,9,9
2,2730000,Delhi,1000,Noida,3,1,9,9,9,9,...,9,9,9,9,9,9,9,9,9,9
3,11100000,Mumbai,655,Andheri East,2,1,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,5000000,Chennai,1225,Maraimalai Nagar,3,0,0,1,1,0,...,1,0,0,0,0,0,0,0,0,0


In [7]:
train_df['price_per_foot^2'] = train_df['Price']/train_df['Area']

In [8]:
train_df.head()

,Price,city,Area,Location,No. of Bedrooms,Resale,MaintenanceStaff,Gymnasium,SwimmingPool,LandscapedGardens,...,BED,VaastuCompliant,Microwave,GolfCourse,TV,DiningTable,Sofa,Wardrobe,Stadium,price_per_foot^2
0,11000000,Delhi,1400,Sector 11 Dwarka,2,1,0,0,0,0,...,1,0,1,0,1,1,1,0,1,7857.142857
1,4883000,Bangalore,1565,Varthur,3,0,9,9,9,9,...,9,9,9,9,9,9,9,9,9,3120.127796
2,2730000,Delhi,1000,Noida,3,1,9,9,9,9,...,9,9,9,9,9,9,9,9,9,2730.000000
3,11100000,Mumbai,655,Andheri East,2,1,1,0,0,0,...,0,0,0,0,0,0,0,0,0,16946.564885
4,5000000,Chennai,1225,Maraimalai Nagar,3,0,0,1,1,0,...,0,0,0,0,0,0,0,0,0,4081.632653


In [9]:
df_summary = train_df.groupby(['city','Location'])[['price_per_foot^2']].mean()

In [10]:
df_summary.shape

(1477, 1)

In [11]:
train_df['Location'].nunique()

1469

In [12]:
train_df.shape

(20974, 42)

In [13]:
# creamos un var donde eliminamos espacios y digitos y pasamos a minusculas location
train_df['Standard_Location'] = train_df['Location'].str.lower().str.replace(' ','').str.replace(r'\d+','', regex=True)

In [14]:
train_df['Standard_Location'] = train_df['Standard_Location'].str.replace('sector','')

In [15]:
train_df['Standard_Location'].nunique()

1346

In [16]:
df_summary_2 = train_df.groupby(['city','Standard_Location'])[['price_per_foot^2']].mean()

In [17]:
city_location_dict = df_summary_2.to_dict()

In [18]:
city_location_dict = city_location_dict['price_per_foot^2']

In [19]:
city_location_dict['Bangalore', 'abbigere']

4637.350946925415

In [20]:
def predict_base_model(data):
    result = data.apply(
        func=lambda x: city_location_dict[x['city'], x['Standard_Location']]*x['Area'], 
        axis=1)
    return result.values

In [21]:
def extend_features(data):
    data.loc[:,'Standard_Location'] = ( data['Location'].
                                       str.lower().
                                       str.replace(' ','').
                                       str.replace(r'\d+','', regex=True).
                                       str.replace('sector','') )
    return data

In [22]:
train_df_eval = train_df.iloc[:2]
train_df_eval_extent = extend_features(train_df.iloc[:2])
predict_base_model(train_df_eval_extent)

array([13656822.08601393,  9568386.29637858])

In [23]:
train_df_eval['Price']

0    11000000
1     4883000
Name: Price, dtype: int64

In [24]:
TEST_DATA = "data/preprocessed/test_data.csv"
test_data = pd.read_csv(TEST_DATA)
test_data_extent = extend_features(test_data)
predict_base_model(test_data_extent)


KeyError: ('Mumbai', 'yogihills')

In [25]:
df_summary_city = train_df.groupby(['city'])[['price_per_foot^2']].mean()

In [26]:
df_summary_city

,price_per_foot^2
city,
Bangalore,6932.804179
Chennai,7019.362670
Delhi,11506.932890
Hyderabad,5458.042928
Kolkata,6777.698922
Mumbai,15846.376865


In [27]:
city_average_dict = df_summary_city.to_dict()['price_per_foot^2']

In [28]:
city_average_dict

{'Bangalore': 6932.804178716791,
 'Chennai': 7019.362670066312,
 'Delhi': 11506.932890232265,
 'Hyderabad': 5458.0429275087,
 'Kolkata': 6777.69892197187,
 'Mumbai': 15846.376864575315}

In [29]:
def predict_robust_base_model(data):
    def calculate_price(x):
        city_average = city_average_dict[x['city']]
        p_p_f = city_location_dict.get((x['city'], x['Standard_Location']),city_average)
        return p_p_f*x['Area']
    
    result = data.apply(
        func=calculate_price, 
        axis=1)
    return result.values

In [30]:
predict_robust_base_model(test_data_extent)

array([ 6139779.06481275,  8216574.07930168,  5896472.2183371 , ...,
        2507638.09815173, 10949324.81253514,  7902346.82593215])

In [31]:
from sklearn.metrics import ( root_mean_squared_error, 
                             mean_absolute_error, 
                             mean_absolute_percentage_error )

In [32]:
y_true = test_data['Price'].values
y_pred = predict_robust_base_model(test_data_extent)
rmse = root_mean_squared_error(y_true, y_pred)
mae = mean_absolute_error(y_true, y_pred)
mape = mean_absolute_percentage_error(y_true, y_pred)
print(f"rmse: {rmse}")
print(f"mae: {mae}")
print(f"mape: {mape}")

rmse: 28193630.458925344
mae: 7708281.0879643895
mape: 0.873278109175525
